Stanisław Władyka 114114

https://github.com/swladyk/Quantum-Machine-Learning-SGH.git


## Wyniki Eksperymentu

### Konfiguracja
- **Ticker:** `NVDA`
- **Okres:** 2020-01-01 do 2025-01-01
- **Okno czasowe:** 20 dni
- **Epoki:** 100
- **Batch size:** 32
- **Learning rate:** 0.01
- **Seed:** 42

### Finalne wyniki na zbiorze testowym

| Model | Accuracy | Precision (UP) | Recall (UP) | Parametry (wagi) |
|-------|----------|----------------|-------------|-----------|
| **QTSA (Quantum)** | **57.7%** | 62.2% | 60.9% | 63 |
| **MLP (Classical)** | **57.7%** | 59.4% | 75.4% | ~1,100 |


In [ ]:
from __future__ import annotations

# ---
# jupyter:
#   jupytext:
#     formats: ipynb,py:percent
#     text_representation:
#       extension: .py
#       format_name: percent
#       format_version: '1.3'
#       jupytext_version: 1.18.1
#   kernelspec:
#     display_name: .venv (3.13.11)
#     language: python
#     name: python3
# ---

# QTSA: Binary Classification - Quantum vs Classical

## Professional Financial ML Approach for SGH Final Project

**Problem:** Predicting market direction (Up/Down) as a **Binary Classification Task**.

### Why Classification, not Regression?

In professional Financial ML, we **DO NOT** predict raw prices/returns because:
1. **Non-stationarity:** Asset prices are non-stationary random walks
2. **Noise fitting:** Regression on prices leads to overfitting to market noise
3. **Business value:** Traders care about **direction** (buy/sell signal), not exact price

### Our Approach:
- **Feature:** Stationary log-returns, normalized to [0, π] for quantum gates
- **Target:** Binary label: `1` if next return > 0 (market goes UP), else `0` (DOWN)
- **Loss:** Binary Cross-Entropy (BCE)
- **Metrics:** Accuracy, Confusion Matrix
- **Models:** QTSA (1-qubit Serial Re-uploading) vs Classical MLP

In [ ]:
import math
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pennylane as qml
import seaborn as sns
import torch
import yfinance as yf
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import RobustScaler
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm

## Configuration

In [ ]:
TICKER = "NVDA"  # Nvidia - strong trends, good for classification
START_DATE = "2020-01-01"
END_DATE = "2025-01-01"
WINDOW_SIZE = 20  # 20 days of historical log-returns
EPOCHS = 100
BATCH_SIZE = 32
LR = 0.01
SEED = 42

torch.manual_seed(SEED)
np.random.seed(SEED)

print(f"Configuration:")
print(f"  Ticker: {TICKER}")
print(f"  Date range: {START_DATE} to {END_DATE}")
print(f"  Window size: {WINDOW_SIZE} days")
print(f"  Task: Binary Classification (Market Direction)")

## Quantum Architecture: Serial Data Re-uploading for Classification

### Adapted QTSA for Binary Classification

The quantum circuit remains the same (serial re-uploading), but the **output interpretation** changes:

```
|0⟩ ─── Rot(θ₀) ─── RX(x₀) ─── Rot(θ₁) ─── RX(x₁) ─── ... ─── Rot(θ_L) ─── ⟨Z⟩
```

**Key change:** Map ⟨Z⟩ ∈ [-1, 1] to probability P(UP) ∈ [0, 1]:

```
P(market goes UP) = (⟨Z⟩ + 1) / 2
```

- ⟨Z⟩ = +1 → P(UP) = 1.0 (100% confident: market goes UP)
- ⟨Z⟩ = -1 → P(UP) = 0.0 (100% confident: market goes DOWN)
- ⟨Z⟩ = 0  → P(UP) = 0.5 (uncertain)

### Comparison: Classification Task

| Aspect | QTSA (Quantum) | MLP (Classical) |
|--------|----------------|-----------------|
| **Task** | Binary Classification | Binary Classification |
| **Architecture** | 1 qubit, serial re-uploading | 3-layer FC with Sigmoid |
| **Parameters** | 63 (L=20) | ~1,100 |
| **Output** | Probability from ⟨Z⟩ | Probability from Sigmoid |
| **Loss** | BCE Loss | BCE Loss |
| **Metric** | Accuracy, Confusion Matrix | Accuracy, Confusion Matrix |

### Visualization: Quantum Circuit

In [ ]:
def draw_quantum_circuit(window_size_demo: int = 5):
    """Visualize QTSA circuit for demonstration."""
    # Use local generator to avoid affecting global random state
    generator = torch.Generator().manual_seed(999)
    
    dev = qml.device("default.qubit", wires=1)
    
    @qml.qnode(dev)
    def demo_circuit(inputs, weights):
        for i in range(window_size_demo):
            qml.Rot(weights[i, 0], weights[i, 1], weights[i, 2], wires=0)
            qml.RX(inputs[i], wires=0)
        qml.Rot(weights[-1, 0], weights[-1, 1], weights[-1, 2], wires=0)
        return qml.expval(qml.PauliZ(0))
    
    dummy_inputs = torch.tensor([0.5, 1.0, 1.5, 2.0, 2.5], dtype=torch.float32)
    dummy_weights = torch.randn(window_size_demo + 1, 3, dtype=torch.float32, generator=generator)
    
    _ = demo_circuit(dummy_inputs, dummy_weights)
    
    fig, ax = qml.draw_mpl(demo_circuit, expansion_strategy="device")(dummy_inputs, dummy_weights)
    plt.tight_layout()
    plt.title(f"QTSA Circuit for Classification (L={window_size_demo})")
    return fig

try:
    fig_circuit = draw_quantum_circuit(window_size_demo=5)
    plt.show()
    print(f"✓ Circuit visualization for {5} timesteps (simplified)")
    print(f"✓ Actual model uses WINDOW_SIZE={WINDOW_SIZE}")
except Exception as e:
    print(f"⚠ Cannot draw circuit: {e}")
    print("  (Normal if no graphical backend available)")

## Data Download

In [ ]:
def download_close_prices(ticker: str, start: str, end: str) -> pd.Series:
    """Download closing prices from Yahoo Finance."""
    print(f"Downloading {ticker} data...")
    df = yf.download(ticker, start=start, end=end, progress=False)
    if df.empty or "Close" not in df:
        raise ValueError(f"No Close data for ticker {ticker}.")
    prices = df["Close"].dropna()
    
    # Handle multi-level columns from yfinance (convert to Series if needed)
    if isinstance(prices, pd.DataFrame):
        prices = prices.squeeze()
    
    print(f"  ✓ Downloaded {len(prices)} data points")
    return prices


prices = download_close_prices(TICKER, START_DATE, END_DATE)
print(f"\nPrice statistics:")
print(prices.describe())

## Feature Engineering: Professional Financial ML

### Step 1: Compute Log-Returns (Stationary Feature)

Log-returns are stationary and commonly used in quantitative finance:

```
r_t = log(P_t / P_{t-1})
```

### Step 2: Normalize to [0, π] for Quantum Encoding

We use **RobustScaler** (not MinMaxScaler) to handle outliers:
- RobustScaler uses median and IQR, robust to extreme market events
- Then clip to [0, π] range required by RX gates

In [ ]:
def compute_log_returns(prices: pd.Series) -> pd.Series:
    """Compute log-returns: r_t = log(P_t / P_{t-1})"""
    returns = np.log(prices / prices.shift(1)).dropna()
    print(f"Log-returns computed: {len(returns)} points")
    print(f"  Mean: {float(returns.mean()):.6f}")
    print(f"  Std: {float(returns.std()):.6f}")
    print(f"  Min: {float(returns.min()):.6f}, Max: {float(returns.max()):.6f}")
    return returns


def normalize_to_pi_range(returns: pd.Series) -> Tuple[np.ndarray, RobustScaler]:
    """
    Normalize returns to [0, π] using RobustScaler.
    RobustScaler is better than MinMaxScaler for financial data (outlier-robust).
    """
    scaler = RobustScaler()
    values = returns.values.reshape(-1, 1)
    
    # Scale using median and IQR
    scaled = scaler.fit_transform(values).flatten()
    
    # Map to [0, π] range: shift to positive, then scale
    # Use percentile clipping to handle extreme outliers
    p_low, p_high = np.percentile(scaled, [1, 99])
    scaled_clipped = np.clip(scaled, p_low, p_high)
    
    # Normalize to [0, π]
    min_val = scaled_clipped.min()
    max_val = scaled_clipped.max()
    normalized = (scaled_clipped - min_val) / (max_val - min_val + 1e-9) * np.pi
    
    print(f"Normalization to [0, π]:")
    print(f"  Range: [{float(normalized.min()):.4f}, {float(normalized.max()):.4f}]")
    print(f"  Mean: {float(normalized.mean()):.4f}, Std: {float(normalized.std()):.4f}")
    
    return normalized.astype(np.float32), scaler


log_returns = compute_log_returns(prices)
normalized_returns, scaler = normalize_to_pi_range(log_returns)

## Target Engineering: Binary Labels

**Target:** Predict if the NEXT return will be positive (UP) or negative (DOWN)

```
y_t = 1  if  r_{t+1} > 0  (market goes UP)
y_t = 0  if  r_{t+1} ≤ 0  (market goes DOWN)
```

In [ ]:
def create_binary_target(returns: pd.Series) -> np.ndarray:
    """
    Create binary classification target:
    1 if next return > 0 (UP), 0 otherwise (DOWN)
    """
    # Shift returns backward by 1 to get "next day's return"
    next_returns = returns.shift(-1).dropna()
    binary_target = (next_returns > 0).astype(np.float32).values
    
    print(f"Binary target created:")
    print(f"  Total samples: {len(binary_target)}")
    print(f"  UP (1): {float(binary_target.sum()):.0f} ({float(binary_target.mean()):.1%})")
    print(f"  DOWN (0): {float((1-binary_target).sum()):.0f} ({float((1-binary_target.mean())):.1%})")
    
    return binary_target


# Need to align: normalized_returns and binary_target must match in length
# binary_target is for predicting the NEXT return
binary_target = create_binary_target(log_returns)

# Align lengths: remove last element from normalized_returns (no future target for it)
normalized_returns = normalized_returns[:len(binary_target)]

print(f"\nFinal aligned data:")
print(f"  Features (normalized returns): {len(normalized_returns)}")
print(f"  Targets (binary labels): {len(binary_target)}")

## Sequence Building: Sliding Window

Create sequences of length `WINDOW_SIZE`:
- **X:** [r_{t-L}, r_{t-L+1}, ..., r_{t-1}] (normalized)
- **y:** Binary label for r_t (1=UP, 0=DOWN)

In [ ]:
def build_classification_sequences(
    features: np.ndarray, targets: np.ndarray, window_size: int
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Build sliding window sequences for classification.
    
    Args:
        features: Normalized log-returns [N]
        targets: Binary labels [N]
        window_size: Length of input sequence
        
    Returns:
        X: [N-window_size, window_size]
        y: [N-window_size] binary labels
    """
    if len(features) != len(targets):
        raise ValueError("Features and targets must have same length")
    
    if len(features) <= window_size:
        raise ValueError("Not enough data for given window_size")
    
    X_seq, y_seq = [], []
    for idx in range(window_size, len(features)):
        X_seq.append(features[idx - window_size : idx])
        y_seq.append(targets[idx])
    
    X_seq = np.stack(X_seq)
    y_seq = np.array(y_seq, dtype=np.float32)
    
    print(f"Sequences built:")
    print(f"  X shape: {X_seq.shape}")
    print(f"  y shape: {y_seq.shape}")
    print(f"  Class balance in y: UP={float(y_seq.mean()):.1%}, DOWN={float(1-y_seq.mean()):.1%}")
    
    return X_seq, y_seq


X_all, y_all = build_classification_sequences(normalized_returns, binary_target, WINDOW_SIZE)

# Split chronologically (80/20)
split_idx = int(len(X_all) * 0.8)
X_train, y_train = X_all[:split_idx], y_all[:split_idx]
X_test, y_test = X_all[split_idx:], y_all[split_idx:]

print(f"\nTrain/Test split:")
print(f"  Train: {len(X_train)} samples, UP={float(y_train.mean()):.1%}")
print(f"  Test:  {len(X_test)} samples, UP={float(y_test.mean()):.1%}")

train_loader = DataLoader(
    TensorDataset(torch.tensor(X_train), torch.tensor(y_train)),
    batch_size=BATCH_SIZE,
    shuffle=False,  # Keep chronological order for time series
)
test_loader = DataLoader(
    TensorDataset(torch.tensor(X_test), torch.tensor(y_test)),
    batch_size=BATCH_SIZE,
    shuffle=False,
)

## Quantum Model: QTSA for Binary Classification

In [ ]:
class QTSAClassifier(torch.nn.Module):
    """
    QTSA model adapted for binary classification.
    Output: Probability P(market goes UP) ∈ [0, 1]
    """
    
    def __init__(self, window_size: int, device_name: str = "default.qubit"):
        super().__init__()
        self.window_size = window_size
        self.dev = qml.device(device_name, wires=1)
        weight_shapes = {"weights": (window_size + 1, 3)}
        self.qnode = self._build_qnode()
        self.qlayer = qml.qnn.TorchLayer(self.qnode, weight_shapes)

    def _build_qnode(self):
        window_size = self.window_size

        @qml.qnode(self.dev, interface="torch", diff_method="backprop")
        def qtsa_circuit(inputs, weights):
            # Serial data re-uploading: Rot → RX for each timestep
            for i in range(window_size):
                qml.Rot(weights[i, 0], weights[i, 1], weights[i, 2], wires=0)
                qml.RX(inputs[..., i], wires=0)
            # Final variational layer
            qml.Rot(weights[-1, 0], weights[-1, 1], weights[-1, 2], wires=0)
            return qml.expval(qml.PauliZ(0))

        return qtsa_circuit

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass: return probability P(UP).
        
        ⟨Z⟩ ∈ [-1, 1] → P(UP) = (⟨Z⟩ + 1) / 2 ∈ [0, 1]
        """
        if x.ndim == 1:
            x = x.unsqueeze(0)
        x = x.to(dtype=torch.float32)
        z = self.qlayer(x)  # ⟨Z⟩ ∈ [-1, 1]
        prob_up = (z + 1.0) * 0.5  # Map to [0, 1]
        return prob_up.squeeze()


print("QTSA Classifier Architecture:")
print(f"  Input: {WINDOW_SIZE} normalized log-returns ∈ [0, π]")
print(f"  Circuit: Serial re-uploading with {WINDOW_SIZE+1} Rot layers")
print(f"  Output: P(market UP) ∈ [0, 1]")
print(f"  Parameters: {(WINDOW_SIZE+1) * 3} trainable rotation angles")

## Classical Model: MLP Classifier

In [ ]:
class MLPClassifier(torch.nn.Module):
    """
    Classical MLP baseline for binary classification.
    Output: Probability P(market goes UP) via Sigmoid.
    """
    
    def __init__(self, window_size: int):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(window_size, 64),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(64, 32),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(32, 1),
            torch.nn.Sigmoid(),  # Output probability
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if x.ndim == 1:
            x = x.unsqueeze(0)
        x = x.to(dtype=torch.float32)
        return self.net(x).squeeze(-1)


# Count parameters
mlp_params = sum(p.numel() for p in MLPClassifier(WINDOW_SIZE).parameters())
qtsa_params = (WINDOW_SIZE + 1) * 3

print("MLP Classifier Architecture:")
print(f"  Input: {WINDOW_SIZE} features")
print(f"  Hidden: 64 → 32 neurons with ReLU + Dropout")
print(f"  Output: P(market UP) via Sigmoid")
print(f"  Parameters: {mlp_params}")
print(f"\nParameter comparison: QTSA={qtsa_params} vs MLP={mlp_params} ({mlp_params/qtsa_params:.1f}x more)")

## Training Function with Accuracy Tracking

In [ ]:
def train_classifier(
    model: torch.nn.Module,
    train_loader: DataLoader,
    test_loader: DataLoader,
    epochs: int,
    lr: float,
    model_name: str,
) -> Tuple[List[float], List[float], List[float], List[float]]:
    """
    Train binary classification model.
    
    Returns:
        train_loss_history, test_loss_history, train_acc_history, test_acc_history
    """
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = torch.nn.BCELoss()  # Binary Cross-Entropy Loss
    
    train_loss_hist = []
    test_loss_hist = []
    train_acc_hist = []
    test_acc_hist = []

    for epoch in range(1, epochs + 1):
        # Training
        model.train()
        total_loss = 0.0
        correct = 0
        total = 0
        
        pbar = tqdm(train_loader, desc=f"[{model_name}] Epoch {epoch}/{epochs}", leave=False)
        for xb, yb in pbar:
            xb = xb.to(dtype=torch.float32)
            yb = yb.to(dtype=torch.float32)
            
            optimizer.zero_grad()
            probs = model(xb)  # Probabilities
            loss = loss_fn(probs, yb)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item() * len(xb)
            preds = (probs > 0.5).float()  # Binary predictions
            correct += (preds == yb).sum().item()
            total += len(xb)
            
            pbar.set_postfix(loss=loss.item(), acc=correct/total)

        train_loss = total_loss / total
        train_acc = correct / total

        # Testing
        model.eval()
        total_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for xb, yb in test_loader:
                xb = xb.to(dtype=torch.float32)
                yb = yb.to(dtype=torch.float32)
                probs = model(xb)
                loss = loss_fn(probs, yb)
                total_loss += loss.item() * len(xb)
                preds = (probs > 0.5).float()
                correct += (preds == yb).sum().item()
                total += len(xb)
        
        test_loss = total_loss / total
        test_acc = correct / total

        train_loss_hist.append(train_loss)
        test_loss_hist.append(test_loss)
        train_acc_hist.append(train_acc)
        test_acc_hist.append(test_acc)
        
        if epoch % 10 == 0 or epoch == 1:
            print(
                f"[{model_name}] Epoch {epoch}/{epochs} | "
                f"Train Loss={train_loss:.4f} Acc={train_acc:.3f} | "
                f"Test Loss={test_loss:.4f} Acc={test_acc:.3f}"
            )

    return train_loss_hist, test_loss_hist, train_acc_hist, test_acc_hist

## Training: QTSA Model

In [ ]:
print("=" * 60)
print("Training QTSA (Quantum) Classifier")
print("=" * 60)

qtsa_model = QTSAClassifier(window_size=WINDOW_SIZE)
q_train_loss, q_test_loss, q_train_acc, q_test_acc = train_classifier(
    qtsa_model, train_loader, test_loader, epochs=EPOCHS, lr=LR, model_name="QTSA"
)

print(f"\n✓ QTSA Training Complete")
print(f"  Final Train Accuracy: {q_train_acc[-1]:.3f}")
print(f"  Final Test Accuracy:  {q_test_acc[-1]:.3f}")

## Training: Classical MLP

In [ ]:
print("\n" + "=" * 60)
print("Training Classical MLP Classifier")
print("=" * 60)

mlp_model = MLPClassifier(window_size=WINDOW_SIZE)
cl_train_loss, cl_test_loss, cl_train_acc, cl_test_acc = train_classifier(
    mlp_model, train_loader, test_loader, epochs=EPOCHS, lr=LR, model_name="MLP"
)

print(f"\n✓ MLP Training Complete")
print(f"  Final Train Accuracy: {cl_train_acc[-1]:.3f}")
print(f"  Final Test Accuracy:  {cl_test_acc[-1]:.3f}")

## Predictions and Evaluation

In [ ]:
def get_predictions(model: torch.nn.Module, loader: DataLoader) -> Tuple[np.ndarray, np.ndarray]:
    """
    Get predictions (probabilities and binary labels) from model.
    """
    model.eval()
    all_probs = []
    all_labels = []
    
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(dtype=torch.float32)
            probs = model(xb)
            all_probs.append(probs.cpu().numpy())
            all_labels.append(yb.cpu().numpy())
    
    probs = np.concatenate(all_probs)
    labels = np.concatenate(all_labels)
    preds = (probs > 0.5).astype(int)
    
    return preds, labels


qtsa_preds, qtsa_labels = get_predictions(qtsa_model, test_loader)
mlp_preds, mlp_labels = get_predictions(mlp_model, test_loader)

qtsa_acc = accuracy_score(qtsa_labels, qtsa_preds)
mlp_acc = accuracy_score(mlp_labels, mlp_preds)

print(f"\nFinal Test Set Accuracy:")
print(f"  QTSA (Quantum): {qtsa_acc:.3f} ({qtsa_acc:.1%})")
print(f"  MLP (Classical): {mlp_acc:.3f} ({mlp_acc:.1%})")

## Visualization: Results Comparison

### 1. Confusion Matrices
### 2. Accuracy Comparison
### 3. Training History (Loss & Accuracy)

In [ ]:
def plot_results(
    q_preds, q_labels, q_acc,
    cl_preds, cl_labels, cl_acc,
    q_train_loss, q_test_loss, q_train_acc, q_test_acc,
    cl_train_loss, cl_test_loss, cl_train_acc, cl_test_acc,
):
    """
    Comprehensive visualization for classification results.
    """
    fig = plt.figure(figsize=(16, 10))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
    
    # === ROW 1: Confusion Matrices ===
    ax1 = fig.add_subplot(gs[0, 0])
    cm_qtsa = confusion_matrix(q_labels, q_preds)
    sns.heatmap(cm_qtsa, annot=True, fmt='d', cmap='Blues', ax=ax1,
                xticklabels=['DOWN', 'UP'], yticklabels=['DOWN', 'UP'])
    ax1.set_title(f'QTSA Confusion Matrix\nAccuracy: {q_acc:.3f}', fontsize=12, fontweight='bold')
    ax1.set_ylabel('True Label')
    ax1.set_xlabel('Predicted Label')
    
    ax2 = fig.add_subplot(gs[0, 1])
    cm_mlp = confusion_matrix(cl_labels, cl_preds)
    sns.heatmap(cm_mlp, annot=True, fmt='d', cmap='Oranges', ax=ax2,
                xticklabels=['DOWN', 'UP'], yticklabels=['DOWN', 'UP'])
    ax2.set_title(f'MLP Confusion Matrix\nAccuracy: {cl_acc:.3f}', fontsize=12, fontweight='bold')
    ax2.set_ylabel('True Label')
    ax2.set_xlabel('Predicted Label')
    
    # === ROW 1, COL 3: Accuracy Bar Chart ===
    ax3 = fig.add_subplot(gs[0, 2])
    models = ['QTSA\n(Quantum)', 'MLP\n(Classical)']
    accuracies = [q_acc, cl_acc]
    colors = ['tab:blue', 'tab:orange']
    bars = ax3.bar(models, accuracies, color=colors, alpha=0.7, edgecolor='black')
    ax3.set_ylim([0, 1])
    ax3.set_ylabel('Test Accuracy', fontsize=11)
    ax3.set_title('Model Comparison', fontsize=12, fontweight='bold')
    ax3.axhline(0.5, color='red', linestyle='--', linewidth=1, label='Random Baseline')
    ax3.legend()
    # Add value labels on bars
    for bar, acc in zip(bars, accuracies):
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                f'{acc:.3f}', ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # === ROW 2: Loss History ===
    ax4 = fig.add_subplot(gs[1, :])
    epochs_range = range(1, len(q_train_loss) + 1)
    ax4.plot(epochs_range, q_train_loss, label='QTSA Train Loss', color='tab:blue', alpha=0.6)
    ax4.plot(epochs_range, q_test_loss, label='QTSA Test Loss', color='tab:blue', linewidth=2)
    ax4.plot(epochs_range, cl_train_loss, label='MLP Train Loss', color='tab:orange', alpha=0.6)
    ax4.plot(epochs_range, cl_test_loss, label='MLP Test Loss', color='tab:orange', linewidth=2)
    ax4.set_xlabel('Epoch', fontsize=11)
    ax4.set_ylabel('Binary Cross-Entropy Loss', fontsize=11)
    ax4.set_title('Training History: Loss', fontsize=12, fontweight='bold')
    ax4.legend(loc='upper right')
    ax4.grid(alpha=0.3)
    
    # === ROW 3: Accuracy History ===
    ax5 = fig.add_subplot(gs[2, :])
    ax5.plot(epochs_range, q_train_acc, label='QTSA Train Acc', color='tab:blue', alpha=0.6)
    ax5.plot(epochs_range, q_test_acc, label='QTSA Test Acc', color='tab:blue', linewidth=2)
    ax5.plot(epochs_range, cl_train_acc, label='MLP Train Acc', color='tab:orange', alpha=0.6)
    ax5.plot(epochs_range, cl_test_acc, label='MLP Test Acc', color='tab:orange', linewidth=2)
    ax5.axhline(0.5, color='red', linestyle='--', linewidth=1, alpha=0.5, label='Random Baseline')
    ax5.set_xlabel('Epoch', fontsize=11)
    ax5.set_ylabel('Accuracy', fontsize=11)
    ax5.set_title('Training History: Accuracy', fontsize=12, fontweight='bold')
    ax5.set_ylim([0.4, 1.0])
    ax5.legend(loc='lower right')
    ax5.grid(alpha=0.3)
    
    plt.suptitle(
        f'QTSA vs Classical MLP: Binary Market Direction Classification\n'
        f'Ticker: {TICKER} | Window: {WINDOW_SIZE} days | Test Samples: {len(q_labels)}',
        fontsize=14, fontweight='bold', y=0.995
    )
    
    plt.show()


plot_results(
    qtsa_preds, qtsa_labels, qtsa_acc,
    mlp_preds, mlp_labels, mlp_acc,
    q_train_loss, q_test_loss, q_train_acc, q_test_acc,
    cl_train_loss, cl_test_loss, cl_train_acc, cl_test_acc,
)

## Summary and Conclusions

### Key Findings:

1. **Professional ML Approach:**
   - We framed the problem as **binary classification** (not regression)
   - Used **stationary features** (log-returns) to avoid non-stationarity issues
   - Evaluated using **accuracy** and **confusion matrices** (not MSE)

2. **Model Comparison:**
   - **QTSA:** Quantum model with only ~63 parameters
   - **MLP:** Classical baseline with ~1,100 parameters (17x more)
   - Both models predict market direction (UP/DOWN) from historical returns

3. **Performance:**
   - Compare test accuracy against random baseline (50%)
   - Check confusion matrices for bias (e.g., always predicting UP)
   - QTSA's advantage: extreme parameter efficiency

4. **Why This Approach is Better:**
   - ✓ Addresses **non-stationarity** (log-returns are stationary)
   - ✓ Focuses on **actionable signal** (direction, not exact price)
   - ✓ Proper **financial ML metrics** (accuracy, not MSE on prices)
   - ✓ Reduces **overfitting risk** (fewer parameters in QTSA)



In [306]:
print("\n" + "=" * 60)
print("ANALYSIS COMPLETE")
print("=" * 60)
print(f"✓ Task: Binary Classification (Market Direction)")
print(f"✓ Ticker: {TICKER}")
print(f"✓ Models: QTSA (Quantum) vs MLP (Classical)")
print(f"✓ Final Test Accuracy:")
print(f"    QTSA: {qtsa_acc:.3f}")
print(f"    MLP:  {mlp_acc:.3f}")
print(f"✓ Visualization: Confusion matrices, accuracy comparison, training history")
print("=" * 60)


ANALYSIS COMPLETE
✓ Task: Binary Classification (Market Direction)
✓ Ticker: NVDA
✓ Models: QTSA (Quantum) vs MLP (Classical)
✓ Final Test Accuracy:
    QTSA: 0.577
    MLP:  0.577
✓ Visualization: Confusion matrices, accuracy comparison, training history


## 📊 Markdown Results Summary

**Poniżej znajduje się podsumowanie wyników w formacie Markdown.**  
**Skopiuj i wklej na początek notebooka jako komórkę Markdown z wynikami eksperymentu.**

In [307]:
print("=" * 80)
print("COPY THE TEXT BELOW AND PASTE AT THE TOP OF NOTEBOOK AS MARKDOWN CELL")
print("=" * 80)
print()

# Oblicz dodatkowe metryki z confusion matrix
cm_qtsa = confusion_matrix(qtsa_labels, qtsa_preds)
cm_mlp = confusion_matrix(mlp_labels, mlp_preds)

qtsa_tn, qtsa_fp, qtsa_fn, qtsa_tp = cm_qtsa.ravel()
mlp_tn, mlp_fp, mlp_fn, mlp_tp = cm_mlp.ravel()

# Precision, Recall dla klasy UP (1)
qtsa_precision = qtsa_tp / (qtsa_tp + qtsa_fp) if (qtsa_tp + qtsa_fp) > 0 else 0
qtsa_recall = qtsa_tp / (qtsa_tp + qtsa_fn) if (qtsa_tp + qtsa_fn) > 0 else 0
mlp_precision = mlp_tp / (mlp_tp + mlp_fp) if (mlp_tp + mlp_fp) > 0 else 0
mlp_recall = mlp_tp / (mlp_tp + mlp_fn) if (mlp_tp + mlp_fn) > 0 else 0

markdown_output = f"""
---

## 🎯 Wyniki Eksperymentu

### Konfiguracja
- **Ticker:** `{TICKER}`
- **Okres:** {START_DATE} do {END_DATE}
- **Okno czasowe:** {WINDOW_SIZE} dni
- **Epoki:** {EPOCHS}
- **Batch size:** {BATCH_SIZE}
- **Learning rate:** {LR}
- **Seed:** {SEED}

### 📈 Finalne wyniki na zbiorze testowym

| Model | Accuracy | Precision (UP) | Recall (UP) | Parametry |
|-------|----------|----------------|-------------|-----------|
| **QTSA (Quantum)** | **{qtsa_acc:.1%}** | {qtsa_precision:.1%} | {qtsa_recall:.1%} | 63 |
| **MLP (Classical)** | **{mlp_acc:.1%}** | {mlp_precision:.1%} | {mlp_recall:.1%} | ~1,100 |
| Random Baseline | 50.0% | - | - | - |

### 🔍 Confusion Matrix - QTSA

|           | Pred DOWN | Pred UP |
|-----------|-----------|---------|
| **True DOWN** | {qtsa_tn} | {qtsa_fp} |
| **True UP**   | {qtsa_fn} | {qtsa_tp} |

### 🔍 Confusion Matrix - MLP

|           | Pred DOWN | Pred UP |
|-----------|-----------|---------|
| **True DOWN** | {mlp_tn} | {mlp_fp} |
| **True UP**   | {mlp_fn} | {mlp_tp} |

### 💡 Wnioski

1. **Efektywność parametryczna:** QTSA osiąga {qtsa_acc:.1%} accuracy z **17× mniejszą liczbą parametrów** niż MLP
2. **Przewaga nad losowym baseline:** Oba modele przewyższają losowe zgadywanie (50%)
3. **Klasyfikacja binarna:** Zastosowanie professional Financial ML approach (log-returns, stationarity)
4. **Architektura kwantowa:** Serial data re-uploading na 1 kubicie skutecznie koduje sekwencje czasowe

---
"""

print(markdown_output)
print("=" * 80)
print("END OF MARKDOWN SUMMARY")
print("=" * 80)

COPY THE TEXT BELOW AND PASTE AT THE TOP OF NOTEBOOK AS MARKDOWN CELL


---

## 🎯 Wyniki Eksperymentu

### Konfiguracja
- **Ticker:** `NVDA`
- **Okres:** 2020-01-01 do 2025-01-01
- **Okno czasowe:** 20 dni
- **Epoki:** 100
- **Batch size:** 32
- **Learning rate:** 0.01
- **Seed:** 42

### 📈 Finalne wyniki na zbiorze testowym

| Model | Accuracy | Precision (UP) | Recall (UP) | Parametry |
|-------|----------|----------------|-------------|-----------|
| **QTSA (Quantum)** | **57.7%** | 62.2% | 60.9% | 63 |
| **MLP (Classical)** | **57.7%** | 59.4% | 75.4% | ~1,100 |
| Random Baseline | 50.0% | - | - | - |

### 🔍 Confusion Matrix - QTSA

|           | Pred DOWN | Pred UP |
|-----------|-----------|---------|
| **True DOWN** | 59 | 51 |
| **True UP**   | 54 | 84 |

### 🔍 Confusion Matrix - MLP

|           | Pred DOWN | Pred UP |
|-----------|-----------|---------|
| **True DOWN** | 39 | 71 |
| **True UP**   | 34 | 104 |

### 💡 Wnioski

1. **Efektywność parametryczna:** QTSA osiąga 57.7%